In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

# Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [3]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [4]:
# 2. Create a empty list to hold the images and titles.
hemisphere_image_urls = []

In [5]:
# 3. Retrieve the image urls and titles for each hemisphere.
# Get a List of All the Hemisphere
product_links = browser.find_by_css("a.product-item h3")
# Using a for loop, iterate through the tags or CSS element
for item in range(len(product_links)):
    #Create an empty dictionary
    hemisphere = {}
    # click on each hemisphere link
    image_browser = browser.find_by_css("a.product-item h3")[item]
    image_browser.click()
    # Navigate to the full-resolution image page & retrieve the full-resolution image URL string
    sample_element = browser.find_link_by_text("Sample").first
    img_url = sample_element["href"]
    # Get Title
    img_title = browser.find_by_css("h2.title").text
    
    # 4 Append Hemisphere dictionary of each image url and title to List
    hemisphere_image_urls.append({"Title": img_title, "Image_Url": img_url})
    
    # Navigate Backwards
    browser.back()

C:\Users\Sungil Baik\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:498: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  warnings.warn(


In [6]:
# Print the list that holds the dictionary of each image title and url.
hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [7]:
# 5. Quit the browser
browser.quit()